In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
# from siuba import *
import numpy as np

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)

In [4]:
from calitp_data_analysis.tables import tbls

In [5]:
from calitp_data_analysis import get_fs

# Test out MPO-provided planned major stops

Here is the schema to use:

| Field name |	Data type |Naming convention for field values |
|-----|-----|-----|
| mpo	| Text |	Use your MPO/RTPA acronym, not full name |
| plan_name	| Text |	Include plan name and year e.g. “2025 mtp” |
| hqta_type	| Text |	Use types from Caltrans major stop map: major_stop_bus, major_stop_rail, major_stop_ferry, major_stop_brt |
| stop_id (optional)	| Text |	None |
| avg_trips_per_peak_hr (optional)	| Float |	None |


In [7]:
path

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/mpo_input/sandag.geojson'

In [8]:
# gpd.read_file(path)

In [9]:
fs = get_fs()

## SANDAG

In [17]:
path = f'{GCS_FILE_PATH}mpo_input/sandag.geojson'

In [18]:
with fs.open(path) as f:
    sandag = gpd.read_file(f)

In [19]:
sandag.head(3)

,OBJECTID,mpo,plan_name,hqta_type,stop_id,avg_trips_per_peak_hr,geometry
0,1,SANDAG,2021RP_2035,major_stop_bus,23740,None,POINT (-117.29703 33.25427)
1,2,SANDAG,2021RP_2035,major_stop_brt,5592,None,POINT (-117.30019 33.25080)
2,3,SANDAG,2021RP_2035,major_stop_bus,5601,None,POINT (-117.29531 33.24924)


## SCAG

In [13]:
path = f'{GCS_FILE_PATH}mpo_input/scag.geojson'

In [15]:
with fs.open(path) as f:
    scag = gpd.read_file(f)

In [16]:
scag.head(3)

,FID,mpo,plan_name,hqta_type,geometry
0,0,SCAG,2024 RTP,major_stop_bus,POINT (-119.17982 34.19901)
1,1,SCAG,2024 RTP,major_stop_bus,POINT (-119.17770 34.16101)
2,2,SCAG,2024 RTP,major_stop_bus,POINT (-118.60602 34.21970)


## SACOG

In [20]:
path = f'{GCS_FILE_PATH}mpo_input/sacog.geojson'

In [21]:
with fs.open(path) as f:
    sacog = gpd.read_file(f)

In [22]:
sacog.head(3)

,OBJECTID,stop_id,mpo,plan_name,hqta_type,avg_trips_per_peak_hr,geometry
0,1,2008,SACOG,2020 MTP-SCS,major_stop_bus,None,POINT (-121.49482 38.58068)
1,2,2020,SACOG,2020 MTP-SCS,major_stop_brt,None,POINT (-121.28658 38.74549)
2,3,2025,SACOG,2020 MTP-SCS,major_stop_bus,None,POINT (-121.54711 38.58500)


## MTC

In [24]:
path = f'{GCS_FILE_PATH}mpo_input/mtc.geojson'

In [25]:
with fs.open(path) as f:
    mtc = gpd.read_file(f)

In [26]:
mtc.head(3)

,FID,primary_ag,hqta_type,mpo,plan_name,geometry
0,0,AC Transit,major_stop_brt,MTC,Plan Bay Area 2050 (2021),POINT (-122.29408 37.87535)
1,1,SamTrans,major_stop_brt,MTC,Plan Bay Area 2050 (2021),POINT (-122.35600 37.57840)
2,2,SamTrans,major_stop_brt,MTC,Plan Bay Area 2050 (2021),POINT (-122.34807 37.57595)


In [29]:
required_cols = ['mpo', 'hqta_type', 'plan_name']

In [31]:
set(required_cols).issubset(mtc.columns)

True

In [30]:
mtc.columns.isin(required_cols)

array([False, False,  True,  True,  True, False])

In [40]:
[col for col in required_cols if col in mtc.columns]

['mpo', 'hqta_type', 'plan_name']

In [41]:
def check_standardize(mpo_gdf):
    required_cols = ['mpo', 'hqta_type', 'plan_name']
    optional_cols = ['stop_id', 'avg_trips_per_peak_hr']
    all_cols = required_cols + optional_cols + ['geometry']
    assert set(required_cols).issubset(mpo_gdf.columns)
    filter_cols = [col for col in all_cols if col in mpo_gdf.columns]
    mpo_gdf = mpo_gdf[filter_cols]
    return mpo_gdf

In [43]:
mpo_planned_mts = pd.concat([check_standardize(gdf) for gdf in [sacog, sandag, scag, mtc]])

In [44]:
mpo_planned_mts

,mpo,hqta_type,plan_name,stop_id,avg_trips_per_peak_hr,geometry
0,SACOG,major_stop_bus,2020 MTP-SCS,2008,None,POINT (-121.49482 38.58068)
1,SACOG,major_stop_brt,2020 MTP-SCS,2020,None,POINT (-121.28658 38.74549)
2,SACOG,major_stop_bus,2020 MTP-SCS,2025,None,POINT (-121.54711 38.58500)
3,SACOG,major_stop_bus,2020 MTP-SCS,2025,None,POINT (-121.54711 38.58500)
4,SACOG,major_stop_bus,2020 MTP-SCS,2068,None,POINT (-121.48656 38.57847)
...,...,...,...,...,...,...
300,MTC,major_stop_brt,Plan Bay Area 2050 (2021),NaN,NaN,POINT (-122.37223 37.58778)
301,MTC,major_stop_rail,Plan Bay Area 2050 (2021),NaN,NaN,POINT (-121.73110 37.70550)
302,MTC,major_stop_rail,Plan Bay Area 2050 (2021),NaN,NaN,POINT (-122.65597 38.26787)
303,MTC,major_stop_brt,Plan Bay Area 2050 (2021),NaN,NaN,POINT (-122.34518 37.96239)


In [45]:
# mpo_planned_mts.explore()